In [1]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
def linear_congruential_generator(N):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = 1

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples

## Test Script:
# N = 10
# print(linear_congruential_generator(N))

In [3]:
def inverse_normal_approximation(u):
    """Generates normally distributed realizations from uniform random realizations."""
    
    ## Constants for approximations to inverse normal.
    a0, a1, a2, a3 = 2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637
    b0, b1, b2, b3 = -8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833
    c0, c1, c2, c3 = 0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863
    c4, c5, c6, c7, c8 = 0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364, 0.0000003960315187

    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4 + r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x

    return x

## Test Script:
# u = 0.8
# print(inverse_normal_approximation(u))
## Results verified here: https://statisticshelper.com/inverse-normal-distribution-calculator/#answer

## Test Script 2:
# N = 1000
# k = 5
# z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
# plt.hist(z)

In [4]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

## Test Script:
# S = 100
# K = 100
# T = 1
# r = 0.03
# q = 0.03
# sigma = 0.2

# print("Call Option Value:", bs_call(S, K, T, r, q, sigma))
# print("Put Option Value:", bs_put(S, K, T, r, q, sigma))
# print("Call Option Delta:", bs_call_delta(S, K, T, r, q, sigma))
# print("Put Option Delta:", bs_put_delta(S, K, T, r, q, sigma))
# print("Option Gamma:", bs_gamma(S, K, T, r, q, sigma))
# print("Option Vega:", bs_vega(S, K, T, r, q, sigma))
## Verified results are correct here: https://goodcalculators.com/black-scholes-calculator/

In [5]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def call_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European call option at time T."""
    return np.exp(-r * T) * np.maximum(Si - K, 0)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

def call_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def put_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def call_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)

def put_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)


## Test Script:
# S0 = 100
# K = 100
# T = 1
# r = 0.03
# q = 0.03
# sigma = 0.2
# zi = 1

# Si = simulated_spot(S0, T, r, q, sigma, zi)
# print("Simulated Spot (Si):", Si)
# print("Call Option Payoff (Ci):", call_payoff(Si, K, T, r))
# print("Put Option Payoff (Pi):", put_payoff(Si, K, T, r))
# print("Call Option Delta Estimate (∆i(C)):", call_delta_estimate(Si, S0, K, T, r))
# print("Put Option Delta Estimate (∆i(P)):", put_delta_estimate(Si, S0, K, T, r))
# print("Call Option Vega Estimate (vegai(C)):", call_vega_estimate(Si, K, T, r, sigma, zi))
# print("Put Option Vega Estimate (vegai(P)):", put_vega_estimate(Si, K, T, r, sigma, zi))

In [6]:
def HW1_NUM3(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Vectorized calculation for call payoff, delta, and vega.
    C_hat_array = call_payoff(Si_array, K, T, r)
    delta_C_hat_array = call_delta_estimate(Si_array, S0, K, T, r)
    vega_C_hat_array = call_vega_estimate(Si_array, K, T, r, sigma, z)

    ## Aggregation.
    N = len(z)
    C_hat = np.mean(C_hat_array)
    delta_C_hat = np.mean(delta_C_hat_array)
    vega_C_hat = np.mean(vega_C_hat_array)

    ## Output results:
    print("Results for N =", N)
    print("Call Option Value -- Monte Carlo:", C_hat)
    print("Call Option Value -- BS:", bs_call(S0, K, T, r, q, sigma))
    print("sqrt(N)*|C_BS - C_hat|:", math.sqrt(N)*abs(bs_call(S0, K, T, r, q, sigma)-C_hat))
    print("Call Option Delta -- Monte Carlo:", delta_C_hat)
    print("Call Option Delta -- BS:", bs_call_delta(S0, K, T, r, q, sigma))
    print("sqrt(N)*|delta_C_BS - delta_C_hat|:", math.sqrt(N)*abs(bs_call_delta(S0, K, T, r, q, sigma)-delta_C_hat))      
    print("Call Option Vega -- Monte Carlo:", vega_C_hat)
    print("Call Option Vega -- BS:", bs_vega(S0, K, T, r, q, sigma))
    print("sqrt(N)*|vega_C_BS - vega_C_hat|:", math.sqrt(N)*abs(bs_vega(S0, K, T, r, q, sigma)-vega_C_hat))
    print("\n")

In [7]:
## Table values for #3.

## Given values.
S0 = 41
K = 42
sigma = 0.25
q = 0.01
r = 0.03
T = 0.75

## Initialize variables. 
N = 10_000

for k in range(10):
    z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
    HW1_NUM3(S0, K, T, r, q, sigma, z)

Results for N = 10000
Call Option Value -- Monte Carlo: 3.364443246828319
Call Option Value -- BS: 3.3411791913217144
sqrt(N)*|C_BS - C_hat|: 2.326405550660482
Call Option Delta -- Monte Carlo: 0.5258680322254572
Call Option Delta -- BS: 0.5224707292291643
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.3397302996292928
Call Option Vega -- Monte Carlo: 14.120530414994855
Call Option Vega -- BS: 14.028613519226251
sqrt(N)*|vega_C_BS - vega_C_hat|: 9.191689576860362


Results for N = 20000
Call Option Value -- Monte Carlo: 3.347003279669108
Call Option Value -- BS: 3.3411791913217144
sqrt(N)*|C_BS - C_hat|: 0.8236504729342926
Call Option Delta -- Monte Carlo: 0.5201341933825683
Call Option Delta -- BS: 0.5224707292291643
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.33043606832269484
Call Option Vega -- Monte Carlo: 14.062738032656215
Call Option Vega -- BS: 14.028613519226251
sqrt(N)*|vega_C_BS - vega_C_hat|: 4.825934970203749


Results for N = 40000
Call Option Value -- Monte Carlo: 3.326600575457776
Ca

In [8]:
## Central Finite Difference Formulas
def call_delta_estimate_FDM(Si_plus, Si_minus, ds, K, T, r):
    """Calculate the delta estimate of the European call option at time T using central finite differences."""
    return (call_payoff(Si_plus, K, T, r) - call_payoff(Si_minus, K, T, r)) / (2 * ds)

def call_gamma_estimate_FDM(Si, Si_plus, Si_minus, ds, K, T, r):
    """Calculate the gamma estimate of the European call option at time T using central finite differences."""
    return (call_payoff(Si_plus, K, T, r) - 2 * call_payoff(Si, K, T, r) + call_payoff(Si_minus, K, T, r)) / (ds**2)

In [9]:
def HW1_NUM4(S0, K, T, r, q, sigma, z):
    ## Initialize variables.
    N = len(z)
    ds = 0.01

    ## Vectorized simulation of spot prices.
    Si = simulated_spot(S0, T, r, q, sigma, z)
    Si_plus = simulated_spot(S0 + ds, T, r, q, sigma, z)
    Si_minus = simulated_spot(S0 - ds, T, r, q, sigma, z)

    ## Vectorized calculation for call delta and gamma.
    delta_C_array = call_delta_estimate_FDM(Si_plus, Si_minus, ds, K, T, r)
    gamma_C_array = call_gamma_estimate_FDM(Si, Si_plus, Si_minus, ds, K, T, r)

    ## Monte Carlo estimates.
    delta_C_hat = np.mean(delta_C_array)
    gamma_C_hat = np.mean(gamma_C_array)

    ## Output results:
    print("Results for N =", N)
    print("Call Option Delta -- FDM:", delta_C_hat)
    print("Call Option Delta -- BS:", bs_call_delta(S0, K, T, r, q, sigma))
    print("sqrt(N)*|delta_C_BS - delta_C_hat|:", np.sqrt(N) * abs(bs_call_delta(S0, K, T, r, q, sigma) - delta_C_hat))      
    print("Call Option Gamma -- FDM:", gamma_C_hat)
    print("Call Option Gamma -- BS:", bs_gamma(S0, K, T, r, q, sigma))
    print("sqrt(N)*|gamma_C_BS - gamma_C_hat|:", np.sqrt(N) * abs(bs_gamma(S0, K, T, r, q, sigma) - gamma_C_hat))
    print("\n")

In [10]:
## Table values for #4.

for k in range(10):
    z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
    HW1_NUM4(S0, K, T, r, q, sigma, z)

Results for N = 10000
Call Option Delta -- FDM: 0.5258801373634344
Call Option Delta -- BS: 0.5224707292291643
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.3409408134270153
Call Option Gamma -- FDM: 0.040142107559633836
Call Option Gamma -- BS: 0.04450878768741225
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 0.43666801277784173


Results for N = 20000
Call Option Delta -- FDM: 0.5201133847415125
Call Option Delta -- BS: 0.5224707292291643
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.33337885456226185
Call Option Gamma -- FDM: 0.03176018985925052
Call Option Gamma -- BS: 0.04450878768741225
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 1.802923994982651


Results for N = 40000
Call Option Delta -- FDM: 0.520843022678328
Call Option Delta -- BS: 0.5224707292291643
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.3255413101672433
Call Option Gamma -- FDM: 0.028532821474306178
Call Option Gamma -- BS: 0.04450878768741225
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 3.1951932426212153


Results for N = 80000
Call Option Delta -- FDM: 0.519969